<a href="https://colab.research.google.com/github/youneseltrach/ML_algo/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this lab, I'll Implement a decision tree from scratch and use it to classify mushrooms as edible or poisonous.

> The process of creating a decision tree involves these steps:
1. Begin with all samples at the root node.
2. Evaluate information gain for all possible feature splits, choose the highest.
3. Divide the data set based on the chosen feature, generating left and right branches.
4. Repeat splitting until the stopping criteria is met."

In [16]:
# Import all the packages that we will need during this assignment
import numpy as np
import matplotlib.pyplot as plt

### 1) Calculate the Entropy
Write a helper function, `compute_entropy`, to calculate the entropy (impurity measure) at a node.\
The entropy is then calculated as

$$H(p) = -p \text{log}_2(p) - (1- p) \text{log}_2(1- p)$$

Where : 𝑝 is the fraction/probabilty of examples that have value = 1 in y

In [17]:
def compute_entropy(y):  

    entropy = 0.
    if len(y) != 0 :

        #calulate the probability of examples that have 1
        p = len(y[y == 1]) / len(y)
        
        if p != 0 and p != 1:
            entropy = -p*np.log2(p)-(1-p)*np.log2(1-p)  # because we have two classes, p the proba if y=1, 1-p: the probe if y=0
        else:
            entropy = 0.

    return entropy

### 2)  Split the dataset

In [18]:
def split_dataset(X, node_indices, feature):
  
    left_indices = []
    right_indices = []

    for i in node_indices:   
       if X[i][feature] == 1: 
           left_indices.append(i)
       else:
           right_indices.append(i)

    return left_indices, right_indices

### 3)  Split the dataset

In [19]:
def compute_information_gain(X, y, node_indices, feature):
  
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node   =  X[node_indices]  ,  y[node_indices]
    X_left, y_left   =  X[left_indices]  ,  y[left_indices]
    X_right, y_right =  X[right_indices] ,  y[right_indices]
    
    # You need to return the following variables correctly
    information_gain = 0
    
    node_entropy =  compute_entropy  (y_node)
    # Your code here to compute the entropy at the left branch
    left_entropy =  compute_entropy  (y_left)
    # Your code here to compute the entropy at the right branch
    right_entropy = compute_entropy (y_right)
    
    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)
    
    weighted_entropy = w_left*left_entropy + w_right*right_entropy
    
    information_gain = node_entropy - weighted_entropy
    
    return information_gain

### 4) Get the best split

In [20]:
def get_best_split(X, y, node_indices):   
     
    # Some useful variables
    num_features = X.shape[1]
    
    # You need to return the following variables correctly
    best_feature = -1
    
    max_info_gain = 0
    
    ### START CODE HERE ###
    
    for feature in range(num_features): 

       # Your code here to compute the information gain from splitting on this feature
       info_gain = compute_information_gain(X,y, node_indices,feature)

       # If the information gain is larger than the max seen so far
       if info_gain > max_info_gain:  
           # Your code here to set the max_info_gain and best_feature
           max_info_gain = info_gain
           best_feature = feature

    ### END CODE HERE ##    
    return best_feature

### 5) Building the tree

In [21]:
# Not graded
tree = [] 

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth): 
   
    # Maximum depth reached - stop splitting 
    if current_depth == max_depth: 
        formatting = " "*current_depth + "-"*current_depth 
        print(formatting, "%s leaf node with indices" % branch_name, node_indices) 
        return 
   
    # Otherwise, get best split and split the data 
    # Get the best feature and threshold at this node 
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth 
    print( "%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature)) 
    
    # Split the dataset at the best feature 
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)  
    tree.append((left_indices, right_indices, best_feature)) 
    
    # continue splitting the left and the right child. Increment current depth 
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1) 
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1) 

### 6) Testing our algorithm

In [22]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [31]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]


In [29]:
tree

[([0, 1, 4, 5, 7], [2, 3, 6, 8, 9], 2),
 ([0, 1, 4, 7], [5], 0),
 ([8], [2, 3, 6, 9], 1)]